In [6]:
import pandas as pd
import numpy as np
import sklearn as skt

In [12]:
class Dog:

    def __init__(self, mybreed):
        self.breed = mybreed

In [14]:
dog_obj = Dog(mybreed="Huskie")

__main__.Dog